In [ ]:
# WELCOME ------------------------------------------------------------------------------

In [ ]:
import numpy as np 
import pandas as pd
from scipy.stats import skew

In [ ]:
train = pd.read_csv('/kaggle/input/predict-volcanic-eruptions-ingv-oe/train.csv')
train.head()

In [ ]:
Test = pd.read_csv('/kaggle/input/predict-volcanic-eruptions-ingv-oe/sample_submission.csv')
Test.head()

In [ ]:
def fill_null(data):
    data = data.fillna(0)
    return data
    
    
def basic_stats(data):
    
    for col in data.columns:
        data[f'{col}_mean'] = data[col].mean()
        data[f'{col}_kurtosis'] = data[col].kurtosis()
        data[f'{col}_sum'] = data[col].sum()
        data[f'{col}_max'] = data[col].max()
        data[f'{col}_min'] = data[col].min()
        data[f'{col}_std'] = data[col].std()
        data[f'{col}_skew'] = data[col].skew()
        data[f'{col}_median'] = data[col].median()
        data[f'{col}_mean'] = data[col].mean()
        data[f'{col}_var'] = data[col].var()
        data[f'{col}_99_quantile'] = data[col].quantile(0.99)
        data[f'{col}_90_quantile'] = data[col].quantile(0.90)
        data[f'{col}_70_quantile'] = data[col].quantile(0.70)
        data[f'{col}_30_quantile'] = data[col].quantile(0.30)
        data[f'{col}_10_quantile'] = data[col].quantile(0.10)
        data[f'{col}_50_quantile'] = data[col].quantile(0.5)
        data[f'{col}_5_quantile'] = data[col].quantile(0.05)
        data[f'{col}_1_quantile'] = data[col].quantile(0.01)
    data = data.iloc[0:1,10:]
    return data



def outlier_treatment(data):
    for col in data.columns:
        ninety_quantile = data[col].quantile(0.99)
        print(ninety_quantile)
        
def train_merge (data,train):
    data['segment_id'] = data['segment_id'].astype('float64')
    final_data = data.merge(train, on='segment_id')
    return final_data

In [ ]:
import glob
data = pd.DataFrame()
for i in (train['segment_id']):
    path = f'/kaggle/input/predict-volcanic-eruptions-ingv-oe/train/{i}.csv'
    df = pd.read_csv(path)
    df = fill_null(df)
    df = basic_stats(df)
    df['segment_id'] = i
    
    data = pd.concat([data,df],axis=0,ignore_index=True)

In [ ]:
# preparing test data 
test_data = pd.DataFrame()
a = 0
for i in (Test['segment_id']):
    path = f'/kaggle/input/predict-volcanic-eruptions-ingv-oe/test/{i}.csv'
    df = pd.read_csv(path)
    df = fill_null(df)
    df = basic_stats(df)
    df['segment_id'] = i
    a+=1
#     print(a)
    test_data = pd.concat([test_data,df],axis=0,ignore_index=True)
    

In [ ]:
# merging data with train and adding time to explode column

train_data = train_merge(data,train)


# to remove outliers

train_data['time_to_eruption'].plot(kind='box')

# seems no outliers we are read to go ahead

In [ ]:
# scaling
from sklearn.preprocessing import StandardScaler
def scaling(X_train, X_test, trial):
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    test_data = sc.transform(trial)
    return X_train, X_test, test_data

def lin_model(X_train, X_test, Y_train):
    lin = LinearRegression()
    lin.fit(X_train, Y_train)
    y_test_predict = lin.predict(X_test)
    return y_test_predict

def decision_model(X_train, X_test, Y_train,trial):
    model = DecisionTreeRegressor(criterion='mae',
                                 max_features='auto',
                                 min_samples_split = 2,
                                 min_samples_leaf = 5)    
    model.fit(X_train, Y_train)
    y_test_predict = model.predict(X_test)
    final_predict = model.predict(trial)
    return y_test_predict,final_predict


def random_model(X_train, X_test, Y_train,trial):
    params = {'n_estimators': 400,
         'min_samples_split': 2,
         'min_samples_leaf': 1,
         'max_features': 'sqrt',
         'max_depth': None,
         'bootstrap': False}
    model = RandomForestRegressor(**params)
    model.fit(X_train, Y_train)
    y_test_predict = model.predict(X_test)
    final_predict = model.predict(trial)
    return y_test_predict,final_predict
    
    
def lgbm_model(X_train, X_test,Y_test, Y_train,trial):
    model = lgbm.LGBMRegressor(bagging_fraction=1.0, bagging_freq=3, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, device='gpu',
              feature_fraction=0.7, gpu_id=0, importance_type='split',
              learning_rate=0.01, max_depth=-1, min_child_samples=5,
              min_child_weight=0.001, min_split_gain=0.3, n_estimators=2500,
              n_jobs=-1, num_leaves=150, objective=None, random_state=1176,
              reg_alpha=1, reg_lambda=0.5, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0,
              tree_method='gpu_hist')
     
    model.fit(X_train, Y_train,
              eval_set=[(X_test, Y_test)], eval_metric='mae',
              verbose=1000, early_stopping_rounds=400)
        
    y_test_predict = model.predict(X_test, num_iteration=model.best_iteration_)
    final_predict = model.predict(trial, num_iteration=model.best_iteration_)
    return y_test_predict,final_predict
    

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

def hypermeter_tunning(X_train, Y_train):
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]
    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}
    

    rf = RandomForestRegressor()
    
    # Random search of parameters, using 3 fold cross validation, 
    # search across 100 different combinations, and use all available cores
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
    # Fit the random search model
    rf_random.fit(X_train, Y_train)
    return rf_random.best_params_

In [ ]:
# preparing data for modelling
X = train_data.drop(columns=['segment_id','time_to_eruption'])
Y = train_data['time_to_eruption']

In [ ]:
trial = test_data.copy()
trial = trial.drop(columns=['segment_id'])

In [ ]:
from sklearn.model_selection import KFold 
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgbm
score = []
final = []
kfold = KFold(n_splits=5, random_state=24, shuffle=True)

for train, test in kfold.split(X):
    X_train, X_test = X.iloc[train], X.iloc[test]
    Y_train, Y_test = Y.iloc[train], Y.iloc[test]
    X_train, X_test, t = scaling(X_train, X_test,trial)
    #------- Hypermeter Tuning ----------You can try to find paramter for radom forest it takes 3 hrs
#     parameters = hypermeter_tunning(X_train, Y_train)
#     break
    #====== MODEL =======================
    y_test_predict,final_predict = random_model(X_train, X_test, Y_train,trial)
    #====================================
    mae = mean_absolute_error(Y_test, y_test_predict)
    print(mae)
    score.append(mae)
    final.append(final_predict)
        
        
average_score = np.mean(score)
print('The average mae is ', average_score)

In [ ]:
# adding our prediction in submission aaray 
import numpy as np
prediction = np.mean(final,0)
Test['time_to_eruption'] = prediction
Test.to_csv('rf_mean.csv', index=False)